In [1]:
# dependencies
import pandas as pd

In [2]:
# seed value for the notebook
from numpy.random import seed
seed(42)

In [3]:
# read data
df = pd.read_csv('./resources/google_10yrs_culr_labelled.csv')
df

,close,upper_shadow,lower_shadow,real_body,candlestick_pattern
0,295.875977,2.216705,2.346191,-1.379822,NO_PATTERN
1,301.046600,0.617676,0.000000,3.920288,NO_PATTERN
2,299.935760,2.022430,2.739716,-1.743469,NO_PATTERN
3,303.397797,0.627655,0.009949,4.483215,NO_PATTERN
4,305.604523,2.455780,0.313843,1.404724,NO_PATTERN
...,...,...,...,...,...
2512,1732.380005,15.609985,3.069946,4.270020,NO_PATTERN
2513,1738.849976,7.150024,5.890015,3.849976,NO_PATTERN
2514,1776.089966,14.638062,5.300049,24.454956,NO_PATTERN
2515,1758.719971,33.719971,31.700073,-29.070068,NO_PATTERN


In [4]:
X = df.drop(['candlestick_pattern'], axis=1)
y = df['candlestick_pattern']
print(X.shape, y.shape)

(2517, 4) (2517,)


In [5]:
labels = list(y.unique())
labels

['NO_PATTERN', 'CDLHAMMER', 'CDLEVENINGSTAR', 'CDLMORNINGSTAR']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
# target category encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
y_categorical = to_categorical(y_encoded)
y_categorical

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [8]:
# split 75/25 (train/test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=1)

In [9]:
# scale with MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
list(X_train_scaled)

[array([0.96738794, 0.18538588, 0.05599396, 0.71965935]),
 array([0.33443784, 0.12360707, 0.06275495, 0.50682548]),
 array([0.33438131, 0.16400733, 0.15583082, 0.46497161]),
 array([0.80520116, 0.09663132, 0.07078494, 0.56105578]),
 array([0.04202763, 0.04561174, 0.0263659 , 0.5449347 ]),
 array([0.50451686, 0.0068394 , 0.06349456, 0.91464139]),
 array([0.76615467, 0.01823789, 0.03761002, 0.65534076]),
 array([0.17419851, 0.0210709 , 0.00194737, 0.64985185]),
 array([0.60885029, 0.31028384, 0.24821996, 0.38006349]),
 array([0.12026405, 0.07248663, 0.07251984, 0.50353543]),
 array([1.91544076e-01, 3.03166162e-03, 3.15963916e-04, 5.72358187e-01]),
 array([0.44298131, 0.02912853, 0.02007338, 0.62721294]),
 array([0.08298162, 0.0153933 , 0.02999723, 0.52871863]),
 array([0.04486974, 0.00801203, 0.00589434, 0.58420378]),
 array([0.63637262, 0.01722373, 0.01152559, 0.62511342]),
 array([0.43026321, 0.02583637, 0.00105622, 0.65766598]),
 array([0.20127584, 0.07969513, 0.01411778, 0.56734654])

In [15]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from cnn import SimpleCNN
from series2gaf import GenerateGAF

# -------------------------------------------------------------------
# Generate GAF:

# set parameters
timeSeries = list(X_train_scaled)
windowSize = 50
rollingLength = 10
fileName = './analysis/demo_%02d_%02d'%(windowSize, rollingLength)

# generate GAF pickle file (output by function Numpy.dump)
GenerateGAF(all_ts = timeSeries,
            window_size = windowSize,
            rolling_length = rollingLength,
            fname = fileName)

            
# -------------------------------------------------------------------
# CNN Example:

# using the generated GAF in previous step
# data shape: (15, 50, 50)
gaf = np.load('%s_gaf.pkl'%fileName)
gaf = np.reshape(gaf, (gaf.shape[0], gaf.shape[1], gaf.shape[2], 1))

# the label is consisted of numbers 1, 2 and 3
# label shape: (15, )
cut_point = int(gaf.shape[0]/3)
label = np.zeros(gaf.shape[0])
label[cut_point:cut_point*2] = 1
label[cut_point*2:] = 2
label = to_categorical(label)

# get cnn model ready
# inputs are single channel data: (15, 15, 1)
# output size is 3 because of {1,2,3}-classes
cnn_model = SimpleCNN(input_shape=(gaf.shape[1], gaf.shape[2], 1),
                        output_size=3)

# train the cnn model
train_history = cnn_model.fit(x=gaf, y=label,
                                epochs=10, batch_size=10,
                                validation_split=0.2, verbose=2)

# save trained model
cnn_model.save_model('model_%s.h5'%datetime.strftime(datetime.today(),'%Y%m%d%H%M'))

ImportError: cannot import name 'SimpleCNN' from 'cnn' (/Users/anthonyvanderwal/_python_demo/project3-cnn/cnn.py)